In [ ]:
import sys
sys.path.append("..") # for sibling import

import compyute as cp

In [ ]:
device = "cuda" if cp.engine.gpu_available() else "cpu"
device

# Example 4.2

### Convolutional Neural Network: more complex data

The goal of this model is to classify images of clothing items.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/zalando-research/fashionmnist?resource=download and place it into the *data* directory. Only using the official training data for training, validation and testing, since it is just to showcase the framework.

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd

data = pd.read_csv('../data/fashion_mnist/fashion-mnist_train.csv')
data.head()

In [ ]:
data_tensor = cp.tensor(data.to_numpy())
train, val, test = cp.preprocessing.split_train_val_test(data_tensor, ratio_val=0.02, ratio_test=0.02)

X_train, y_train = train[:, 1:], train[:, 0].int()
X_val, y_val = val[:, :-1], val[:, 0].int()
X_test, y_test = test[:, :-1], test[:, 0].int()

X_train = X_train.reshape((X_train.shape[0], 1 , 28, -1)).float()
X_val = X_val.reshape((X_val.shape[0], 1, 28, -1)).float()
X_test = X_test.reshape((X_test.shape[0], 1, 28, -1)).float()

X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

print (f'{X_train.shape=}')
print (f'{y_train.shape=}')

print (f'{X_val.shape=}')
print (f'{y_val.shape=}')

print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

### Step 2: Build the neural network structure

In [ ]:
import compyute.nn as nn
from compyute.nn.layers import *

model = nn.SequentialModel([
    Convolution2d(1, 64, kernel_size=(5, 5), pad="same", use_bias=False), Batchnorm(64), ReLU(),
    MaxPooling2d(kernel_size=(2, 2)),
    Dropout(0.3),
    Convolution2d(64, 128, kernel_size=(3, 3), pad="same", use_bias=False), Batchnorm(128), ReLU(),
    MaxPooling2d(kernel_size=(2, 2)),
    Dropout(0.3),
    Flatten(),
    Linear(7*7*128, 128, use_bias=False), Batchnorm(128), ReLU(),
    Dropout(0.3),
    Linear(128, 10)
])

In [ ]:
model.compile(
    optimizer=nn.optimizers.Adam(3e-4),
    loss_fn=nn.losses.Crossentropy(),
    metric_fn=nn.metrics.accuracy
)
model.to_device(device)

In [ ]:
from compyute.nn.analysis import model_summary
model_summary(model, (1, 28, 28))

### Step 3: Train the model

In [ ]:
epochs = 5
batch_size = 128

train_losses, train_scores, val_losses, val_scores = model.train(
    X_train, y_train, epochs=epochs, batch_size=batch_size, val_data=(X_val, y_val)
)

In [ ]:
traces = {
    "train_loss" : train_losses,
    "train_accuracy" : train_scores
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="train history", x_label="steps", y_label="loss/accuracy")

In [ ]:
traces = {
    "val_losses" : val_losses,
    "val_scores" : val_scores
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="val history", x_label="steps", y_label="loss/accuracy")

### Step 4: Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size)
print(f'loss {loss:.4f}')
print(f'accuracy {accuracy*100:.2f}')

In [ ]:
predictions = model.predict(X_test, batch_size=batch_size)
nn.analysis.plot_confusion_matrix(predictions, y_test, figsize=(5, 5), cmap='Blues')